# Answering Business Questions in SQLight

In this project I will be quering a database called `chinook.db` to answer business questions. 

The database has the following structure:



In [6]:
from IPython.display import Image

Image(url= 'chinook_outline.png', width = 600, height = 600)

## Load Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
sql

 * sqlite:///chinook.db


'Connected: @chinook.db'

## Getting a Sense of all Tables and Data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT *
FROM invoice_line
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [5]:
%%sql
SELECT *
FROM invoice
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [6]:
%%sql
SELECT *
FROM genre
LIMIT 5;

 * sqlite:///chinook.db
Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll


## Task 1: Find out which artists sell the most tracks in the US:

Write a query that returns each genre, with the number of tracks sold in the US in abs numbers and percentages

In [7]:
%%sql
WITH Q1 as
(SELECT
    g.genre_id,
    g.name as genre_name,
    t.track_id
FROM
    genre as g INNER JOIN track as t ON
    g.genre_id = t.genre_id),

Q2 as
(SELECT 
    Q1.genre_id,
    Q1.genre_name,
    Q1.track_id,
    il.invoice_id,
    il.quantity
FROM
    invoice_line as il INNER JOIN Q1 ON
    Q1.track_id = il.track_id),

Q3 as
(SELECT
    Q2.genre_id,
    Q2.genre_name,
    Q2.track_id,
    Q2.invoice_id,
    Q2.quantity,
    i.billing_country
FROM
    invoice as i INNER JOIN Q2 ON
    Q2.invoice_id = i.invoice_id
WHERE i.billing_country = 'USA'),


Q4 as
(SELECT
    Q3.genre_id,
    Q3.genre_name,
    SUM(Q3.quantity) as Tracks_Sold_USA
FROM Q3
GROUP BY Q3.genre_id)


SELECT
    Q4.genre_id,
    Q4.genre_name,
    Tracks_Sold_USA,
    (Tracks_Sold_USA/(SELECT CAST(SUM(Q4.Tracks_Sold_USA) as FLOAT) FROM Q4)
    )as Percentage_Tracks_Sold_USA
FROM Q4
ORDER BY 3 desc

 * sqlite:///chinook.db
Done.


genre_id,genre_name,Tracks_Sold_USA,Percentage_Tracks_Sold_USA
1,Rock,561,0.5337773549000951
4,Alternative & Punk,130,0.12369172216936251
3,Metal,124,0.11798287345385347
14,R&B/Soul,53,0.05042816365366318
6,Blues,36,0.03425309229305423
23,Alternative,35,0.03330161750713606
7,Latin,22,0.02093244529019981
9,Pop,22,0.02093244529019981
17,Hip Hop/Rap,20,0.019029495718363463
2,Jazz,14,0.013320647002854425


## Task 2: Compare Sales Agents

- query that finds the total dollar amount of sales assigned to each sales support agent within the company 
- Add any extra attributes for that employee that you find are relevant to the analysis.
- Write a short statement describing your results, and providing a possible interpretation.

In [13]:
%%sql
SELECT *
FROM employee
LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [15]:
%%sql
WITH 
    support_agent_sales AS
        (SELECT c.support_rep_id,
                ROUND(CAST(SUM(i.total) AS Float),2) total_sales
         FROM invoice i
         INNER JOIN customer c ON c.customer_id = i.customer_id
         GROUP BY c.support_rep_id
        )
SELECT 
    e.employee_id,
    e.first_name||' '||e.last_name employee_name,
    e.hire_date,
    e.title,
    e.email,
    mgr.first_name||' '||mgr.last_name manager_name,
    ss.total_sales
FROM employee e
INNER JOIN support_agent_sales ss ON ss.support_rep_id = e.employee_id
LEFT JOIN employee mgr ON mgr.employee_id = e.reports_to

 * sqlite:///chinook.db
Done.


employee_id,employee_name,hire_date,title,email,manager_name,total_sales
3,Jane Peacock,2017-04-01 00:00:00,Sales Support Agent,jane@chinookcorp.com,Nancy Edwards,1731.51
4,Margaret Park,2017-05-03 00:00:00,Sales Support Agent,margaret@chinookcorp.com,Nancy Edwards,1584.0
5,Steve Johnson,2017-10-17 00:00:00,Sales Support Agent,steve@chinookcorp.com,Nancy Edwards,1393.92


## Task 3: Analyzing Sales Data for Customers from each country

You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

If there is a particular value that you would like to force to the top or bottom of results, you can put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column. Here's an example - let's start by creating a view so we're working with a manageable number of rows:

Next, inside a subquery, we'll select all values from our view and add a sorting column using a case statement, before sorting using that new column in the outer query.

In [10]:
%%sql
SELECT
    first_name,
    count
FROM
    (
    SELECT
        t5.*,
        CASE
            WHEN t5.first_name = "Mark" THEN 1
            ELSE 0
        END AS sort
    FROM top_5_names t5
   )
ORDER BY sort ASC

 * sqlite:///chinook.db
Done.


first_name,count
Frank,2
Wyatt,1
Victor,1
Tim,1
Mark,2


You should be able to adapt this technique into your query to force 'Other' to the bottom of your results. When working through this exercise, you will need multiple subqueries and joins. Imagine you work on a team of data analysts, and write your query so that it will be able to be easily read and understood by your colleagues.

### Actual Task:
Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
For each country, include:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [16]:
%%sql
WITH 
    country_sales AS
    (
        SELECT    
            c.country,
            COUNT(DISTINCT c.customer_id) total_customers,
            COUNT(i.invoice_id) total_sales,
            CAST(SUM(i.total) AS Float) total_sales_value
        FROM invoice i
        INNER JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY
            c.country
    )
SELECT
    country "Country",
    total_customers "Total Customers",    
    ROUND(total_sales_value,2) "Total Sales Value",
    ROUND((total_sales_value / total_customers),2) "Average Customer Sales Value",
    ROUND((total_sales_value / total_sales),2) "Average Order Value"
FROM (
        SELECT 
            CASE 
                WHEN total_customers > 1 THEN
                    country
                ELSE
                    "Other"
            END country,
            CASE WHEN total_customers > 1 THEN
                1
            ELSE
                0
            END sort,
            SUM(total_customers) total_customers,
            SUM(total_sales) total_sales,
            SUM(total_sales_value) total_sales_value
        FROM country_sales
        GROUP BY 1,2
     )
ORDER BY sort desc,3 desc;

 * sqlite:///chinook.db
Done.


Country,Total Customers,Total Sales Value,Average Customer Sales Value,Average Order Value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


## Task 4: Query that categorizes each Invoice as Album Purchase or not and calculates Summary Statististics

Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

- Number of invoices
- Percentage of invoices
- Write one to two sentences explaining your findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies

In [17]:
%%sql
WITH 
    invoice_album_stat AS
        (
            SELECT 
                is_album_purchase, 
                CAST(COUNT(*) AS Float) number_of_invoices
            FROM (
                    WITH 
                        invoice_track_albums AS
                            (SELECT 
                                 il.invoice_id, 
                                 t.album_id, 
                                 t.track_id
                             FROM invoice_line il
                             INNER JOIN track t ON t.track_id = il.track_id
                            )
                    SELECT i.invoice_id,
                           (
                               CASE WHEN 
                                       (
                                        SELECT ita.track_id
                                        FROM invoice_track_albums ita
                                        WHERE ita.invoice_id = i.invoice_id
                                           EXCEPT
                                        SELECT t.track_id
                                        FROM track t
                                        INNER JOIN album a ON a.album_id = t.album_id
                                        WHERE a.album_id IN (SELECT ita.album_id
                                                             FROM invoice_track_albums ita
                                                             WHERE ita.invoice_id = i.invoice_id
                                                            )
                                       ) IS NULL
                                       AND
                                       (
                                        SELECT t.track_id
                                        FROM track t
                                        INNER JOIN album a ON a.album_id = t.album_id
                                        WHERE a.album_id IN (SELECT ita.album_id
                                                             FROM invoice_track_albums ita
                                                             WHERE ita.invoice_id = i.invoice_id
                                                            )
                                           EXCEPT
                                        SELECT ita.track_id
                                        FROM invoice_track_albums ita
                                        WHERE ita.invoice_id = i.invoice_id 
                                       ) IS NULL
                                   THEN 'Y'
                                   ELSE 'N'
                               END
                           ) is_album_purchase
                    FROM invoice i
                )
            GROUP BY is_album_purchase
        )
SELECT 
    CASE
        WHEN is_album_purchase = "Y" THEN "Whole Album"
        ELSE "Individual Tracks"
    END "Purchase Type", 
    number_of_invoices "Number of Invoices",
    ROUND((number_of_invoices 
           / 
           (SELECT SUM(number_of_invoices)
            FROM invoice_album_stat)
          ) * 100
          ,2) "Percentage of Invoices"
FROM invoice_album_stat

 * sqlite:///chinook.db
Done.


Purchase Type,Number of Invoices,Percentage of Invoices
Individual Tracks,500.0,81.43
Whole Album,114.0,18.57
